In [2]:
from transformers import pipeline, AutoTokenizer
import pandas as pd
import json
from random import Random
from torch.utils.data import Dataset
from tqdm import tqdm
import transformers
import traceback
from Annotation import annotate
from copy import deepcopy

In [4]:
transformers.logging.set_verbosity_error()

ITEMS_PER_CONDITION = 10

class PromptDataset(Dataset):
    def __init__(self, prompts):
        self.prompts = prompts
    def __len__(self):
        return len(self.prompts)
    def __getitem__(self, idx):
        return self.prompts[idx]
        
models = [("stefan-it/german-gpt2-larger", 64, -1, None)]

with open("../items/names.json", encoding="utf-8") as nfile:
    namedict = json.load(nfile)
male_names = [name for name in namedict["male"]] * 2
female_names = [name for name in namedict["female"]] * 2
with open("../items/verbs_forced_reference.json", encoding="utf-8") as nfile:
    verbdict = json.load(nfile)
es_verbs = verbdict["es"]
se_verbs = verbdict["se"]

female_shuffled = female_names.copy()
Random(42).shuffle(female_shuffled)
male_shuffled = male_names.copy()
Random(84).shuffle(male_shuffled)
male_pairing = list(zip(male_names, female_shuffled, [False for name in male_names]))
female_pairing = list(zip(female_names, male_shuffled, [True for name in male_names]))

conditions = [
    (2,  es_verbs, female_pairing, "NP1"),
    (3,  es_verbs,   male_pairing, "NP1"),
    (6,  se_verbs, female_pairing, "NP1"),
    (7,  se_verbs,   male_pairing, "NP1"),
    (10, es_verbs, female_pairing, "NP2"),
    (11, es_verbs,   male_pairing, "NP2"),
    (14, se_verbs, female_pairing, "NP2"),
    (15, se_verbs,   male_pairing, "NP2")
]

items_per_condition = []
  
for condition, verbs, pairing, forced_reference in conditions:
    rows = []
    for verbdict in verbs:
        verb, filler, verbclass = verbdict["verb"], verbdict["filler"], verbdict["verbclass"]
        for np1, np2, female in pairing:
            prompt = f"{np1} {verb} {np2}{filler}, weil"
            nrow = {"condition": condition, "type": "Experiment", "prompt": prompt, "NP1": np1, "NP2": np2, 
                    "NP1gender": "f" if female else "m", "verb": verb, "verbclass": verbclass, "forced": forced_reference}
            rows.append(nrow)
    Random(168).shuffle(rows)
    items_per_condition.append(rows)
    
for model_name, batch_size, device, device_map in models:
       
    print(f"now loading: {model_name}")
    model = pipeline("text-generation", model = model_name, device = device, device_map = device_map)
    model.tokenizer.pad_token_id = model.model.config.eos_token_id
    model.tokenizer.padding_side = "left"


now loading: stefan-it/german-gpt2-larger


In [ ]:
male_tokens = list(map(model.tokenizer.encode, [" er", " dieser", " jener", " der"]))
female_tokens = list(map(model.tokenizer.encode, [" sie", " diese", " jene", " die"]))

data = []

for condition in items_per_condition:
    items = deepcopy(condition)
    bar = tqdm(total = ITEMS_PER_CONDITION)
    item_iter = iter(items)
    rows = []
    while bar.n < ITEMS_PER_CONDITION:
        try:
            row = next(item_iter)
            if row["forced"] == "NP1":
                tokenized_name = model.tokenizer.encode(f" {row['NP1']}")
            else:
                tokenized_name = model.tokenizer.encode(f" {row['NP2']}")
            if (row["NP1gender"] == "m" and row["forced"] == "NP1") or (row["NP1gender"] == "f" and row["forced"] == "NP2"):
                print("Yo, waddup")
                forced_tokens = male_tokens + [tokenized_name]
            else:
                print("Cool story pro")
                forced_tokens = female_tokens # + [tokenized_name]
            print(forced_tokens)
            continuation = model(row["prompt"], force_words_ids = [forced_tokens], remove_invalid_values=True, early_stopping = True, do_sample = False, num_beams = 2, max_new_tokens = 12)[0]["generated_text"]
            row["cont"] = continuation[len(row["prompt"]) + 1:]
            res = annotate(row, False)
            print(row["prompt"], row["cont"])
            print(row["forced"], row["NP1gender"], res["Koreferenz"])
            if res["Koreferenz"] == row["forced"]:
                bar.update(1)
                row.update(res)
                rows.append(row)
        except StopIteration:
            print(f"Run out of data in condition {items[0]['condition']}")
            break
    data += rows

exp3 = pd.DataFrame(data, columns = ["condition", "type", "prompt", "cont", "NP1", "NP2", "NP1gender", "verb", "verbclass", "forced", "Koreferenz", "Anaphorische Form"])
exp3.to_csv(f"../data/forced_coreference--{model_name.replace('/', '--')}.csv", sep=";", index=False)

del exp3
del rows
del items
    

 50%|█████████████████████████████████████████▌                                         | 5/10 [00:45<00:45,  9.19s/it]


Cool story pro
[[467], [846], [8924], [286]]
Annika ekelte sich vor Kevin schon eine ganze Weile, weil er sich nicht die Zähne putzen wollte.
Doch dann
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Melanie vergötterte Marcel vom ersten Moment an, weil er sich in sie verliebt hatte.
Doch dann kam alles
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Katharina sorgte sich um Martin seit mehreren Tagen, weil er sich nicht mehr die Zähne putzen konnte.
Er
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Sarah fürchtete Benjamin ein kleines bisschen, weil er sich nicht die Mühe machen wollte, sich mit dem Thema
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Jessica bedauerte Benjamin aus tiefstem Herzen, weil er sich so sehr für die Kinder eingesetzt habe.
„
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Luisa achtete Marco in hohem Grade, weil er sich in die Mannschaft einmischte.
Er war
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Jacqueline

Jessica achtete Benjamin in hohem Grade, weil er sich in die Frau verliebt hatte, die er liebte.
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Alexandra verehrte Jonas seit mehreren Wochen, weil er sich in sie verliebt hatte.
Als sie ihn am
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Vivien verehrte Stefan seit mehreren Wochen, weil er sich in die junge Frau verliebt hatte.
Als er
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Jennifer verehrte Marcel seit mehreren Wochen, weil er sich in sie verliebt hatte.
Doch als er sie
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Melanie verabscheute Marcel schon seit Wochen, weil er sie nicht mochte.
Doch dann kam die Nachricht,
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
Hannah vergötterte Simon vom ersten Moment an, weil er sich in sie verliebt hatte.
Sie war eine sehr
NP1 f NP2
Cool story pro
[[467], [846], [8924], [286]]
